<a href="https://colab.research.google.com/github/fymatsushita/pediatrics/blob/main/Predictive_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lux
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curvey_scores = model.predict_proba(X_test)
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from pprint import pprint
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier


In [ ]:
df.head()
df['Alert'] = df['Alert'].astype(float)
df.info()

In [ ]:
df.describe().T

In [ ]:
p = df.hist(figsize=(10,10))

In [ ]:
print(df.isnull().sum())

In [ ]:
df.shape

In [ ]:
l = sns.pairplot(df, hue='Alert')

In [ ]:
# Grid search
def GridSearchModel(X, y, model, parameters, cv):
  CV_model = GridSearchCV(estimator=model, param_grid = parameters, cv = cv)
  CV_model.fit(X,y)
  CV_model.cv_results_
  print("Best Score:", CV_model.best_score_, " / Best parameters:", CV_model.best_params_)

In [ ]:
# Learning curve
def LearningCurve(X, y, model, cv, train_sizes):

    train_sizes, train_scores, test_scores = learning_curve(model, X, y, cv = cv, n_jobs = 4, 
                                                            train_sizes = train_sizes)

    train_scores_mean = np.mean(train_scores, axis = 1)
    train_scores_std  = np.std(train_scores, axis = 1)
    test_scores_mean  = np.mean(test_scores, axis = 1)
    test_scores_std   = np.std(test_scores, axis = 1)
    
    train_Error_mean = np.mean(1- train_scores, axis = 1)
    train_Error_std  = np.std(1 - train_scores, axis = 1)
    test_Error_mean  = np.mean(1 - test_scores, axis = 1)
    test_Error_std   = np.std(1 - test_scores, axis = 1)

    Scores_mean = np.mean(train_scores_mean)
    Scores_std = np.mean(train_scores_std)
    
    _, y_pred, Accuracy, Error, precision, recall, f1score = ApplyModel(X, y, model)
    
    plt.figure(figsize = (16,4))
    plt.subplot(1,2,1)
    ax1 = Confuse(y, y_pred, classes)
    plt.subplot(1,2,2)
    plt.fill_between(train_sizes, train_Error_mean - train_Error_std,train_Error_mean + train_Error_std, alpha = 0.1,
                     color = "r")
    plt.fill_between(train_sizes, test_Error_mean - test_Error_std, test_Error_mean + test_Error_std, alpha = 0.1, color = "g")
    plt.plot(train_sizes, train_Error_mean, 'o-', color = "r",label = "Training Error")
    plt.plot(train_sizes, test_Error_mean, 'o-', color = "g",label = "Cross-validation Error")
    plt.legend(loc = "best")
    plt.grid(True)
     
    return (model, Scores_mean, Scores_std )

In [ ]:
def ApplyModel(X, y, model):
    
    model.fit(X, y)
    y_pred  = model.predict(X)

    Accuracy = round(np.median(cross_val_score(model, X, y, cv = cv)),2)*100
 
    Error   = 1 - Accuracy
    
    precision = precision_score(y_train, y_pred) * 100
    recall = recall_score(y_train, y_pred) * 100
    f1score = f1_score(y_train, y_pred) * 100
    
    return (model, y_pred, Accuracy, Error, precision, recall, f1score) 

In [ ]:
def Confuse(y, y_pred, classes):
    cnf_matrix = confusion_matrix(y, y_pred)
    
    cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis = 1)[:, np.newaxis]
    c_train = pd.DataFrame(cnf_matrix, index = classes, columns = classes)  

    ax = sns.heatmap(c_train, annot = True, cmap = cmap, square = True, cbar = False, 
                          fmt = '.2f', annot_kws = {"size": 20})
    return(ax, c_train)

In [ ]:
def PrintResults(model, X, y, title):
    
    model, y_pred, Accuracy, Error, precision, recall, f1score = ApplyModel(X, y, model)
    
    _, Score_mean, Score_std = LearningCurve(X, y, model, cv, train_size)
    Score_mean, Score_std = Score_mean*100, Score_std*100
    
    
    print('Scoring Accuracy: %.2f %%'%(Accuracy))
    print('Scoring Mean: %.2f %%'%(Score_mean))
    print('Scoring Standard Deviation: %.4f %%'%(Score_std))
    print("Precision: %.2f %%"%(precision))
    print("Recall: %.2f %%"%(recall))
    print('f1-score: %.2f %%'%(f1score))
    
    Summary = pd.DataFrame({'Model': title,
                       'Accuracy': Accuracy, 
                       'Score Mean': Score_mean, 
                       'Score St Dv': Score_std, 
                       'Precision': precision, 
                       'Recall': recall, 
                       'F1-Score': f1score}, index = [0])
    return (model, Summary)

In [ ]:
classes = ['0','1']
cv = ShuffleSplit(n_splits = 100, test_size = 0.3, random_state = 0)
train_size = np.linspace(.1, 1.0, 15)

In [ ]:
X = df.drop(['Alert','Lactate_pH'],1)
X2 = df.drop(['pCO2', 'pH', 'BIC', 'DOL','Alert', 'Lactate'], 1)
X3 = df.drop(['Alert','Lactate','Lactate_pH'], 1)
y = df['Alert']

X_train, X_test, y_train, y_test = train_test_split (X, y, test_size=0.3, random_state=40, stratify=y)

X_scaled = StandardScaler().fit_transform(X)
X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = train_test_split (X_scaled, y, test_size=0.3, random_state=40, stratify=y)

In [ ]:
cmap = sns.diverging_palette(220,10, as_cmap=True)
model = LogisticRegression(max_iter=200)
model, Summary_LR = PrintResults(model, X_train, y_train, 'Logistic Regression')

y_train_LR = pd.Series(model.predict(X_train), name = "LR")
y_test_LR = pd.Series(model.predict(X_test), name = "LR")

In [ ]:
y_scores = model.predict_proba(X_test)
y_scores = y_scores[:,1]

false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_scores)

def plot_roc_curve(false_positive_rate, true_positive_rate, label=None):
    plt.plot(false_positive_rate, true_positive_rate, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'r', linewidth=4)
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate (FPR)', fontsize=16)
    plt.ylabel('True Positive Rate (TPR)', fontsize=16)

plt.figure(figsize=(14, 7))
plot_roc_curve(false_positive_rate, true_positive_rate)

pred = model.predict_proba(X_test)[:,1]
roc_auc_score(y_test, pred, average='macro', sample_weight=None)

In [ ]:
model = RandomForestClassifier()

params = {'n_estimators':[10,25,100],
          'max_features':[5,8], # Number of max_features cannot be greater than the number of features
          'max_depth':[10,50,None],
          'bootstrap':[True,False]}

clf = GridSearchCV(estimator=model,
                   param_grid=params,
                   scoring='accuracy',
                   verbose=1)

clf.fit(X,y)
clf.best_estimator_
print("Best parameters:", clf.best_params_)
print("Best F1: ", (-clf.best_score_))

# Performance metrics
grid_best = clf.best_estimator_.predict(X_train)
errors = abs(grid_best - y_train)

# Calculate mean absolute percentage error (MAPE)
mape = np.mean(100 * (errors/y_train))

# Calculate and display accuracy
accuracy = 100 - mape

print('The best model from grid-search has an accuracy of', round(accuracy, 2),'%')

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 20, stop = 200, num = 5)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(1, 45, num = 3)]

# Minimum number of samples required to split a node
min_samples_split = [5,10]

# Create the random grid
random_grid = {'n_estimators':n_estimators,
               'max_features':max_features,
               'max_depth':max_depth,
               'min_samples_split':min_samples_split}

pprint(random_grid)

# Random search of parameters, using 3 fold cross validation. Search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = RandomForestClassifier(), param_distributions = random_grid,
                               n_iter = 10, cv = 10, verbose = 2,
                               random_state=42, n_jobs = -1, scoring='neg_mean_squared_error')

rf_random.fit(X_train, y_train)

rf_random.best_estimator_

random_best = rf_random.best_estimator_.predict(X_train)
errors = abs(random_best - y_train)
mape=np.mean(100*(errors/y_train))
accuracy = 100 - mape
print('The best model from the randomized search has an accuracy of', round(accuracy,2),'%')
rf_random.best_params_

In [ ]:
model = RandomForestClassifier(n_estimators = 100, max_depth=50, bootstrap=True, max_features=6)
model,Summary_RF = PrintResults(model, X_train, y_train, 'Random Forest')
y_train_RF = pd.Series(model.predict(X_train), name="RF")
y_test_RF = pd.Series(model.predict(X_test), name = "RF")

In [ ]:
model = RandomForestClassifier(n_estimators = 100, max_depth=None, bootstrap=True, max_features=6)
model, y_pred, Accuracy, Error, precision, recall, f1score = ApplyModel(X_train, y_train, model)
Priority = pd.DataFrame({'Feature': X_train.columns, 'Importance':np.round(model.feature_importances_,3)})
Priority = Priority.sort_values('Importance',ascending=False).set_index('Feature')
print(Priority)

In [ ]:
y_scores = model.predict_proba(X_test)
y_scores = y_scores[:,1]

false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_scores)

def plot_roc_curve(false_positive_rate, true_positive_rate, label=None):
    plt.plot(false_positive_rate, true_positive_rate, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'r', linewidth=4)
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate (FPR)', fontsize=16)
    plt.ylabel('True Positive Rate (TPR)', fontsize=16)

plt.figure(figsize=(14, 7))
plot_roc_curve(false_positive_rate, true_positive_rate)

pred = model.predict_proba(X_test)[:,1]
roc_auc_score(y_test, pred, average='macro', sample_weight=None)

In [ ]:
model = GaussianNB()
model,Summary_GNB = PrintResults(model, X_train, y_train, "GNB")
y_train_GNB = pd.Series(model.predict(X_train), name = "GNB")
y_test_GNB = pd.Series(model.predict(X_test), name = "GNB")
pred = model.predict_proba(X_test)[:,1]

In [ ]:
y_scores = model.predict_proba(X_test)
y_scores = y_scores[:,1]

false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_scores)

def plot_roc_curve(false_positive_rate, true_positive_rate, label=None):
    plt.plot(false_positive_rate, true_positive_rate, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'r', linewidth=4)
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate (FPR)', fontsize=16)
    plt.ylabel('True Positive Rate (TPR)', fontsize=16)

plt.figure(figsize=(14, 7))
plot_roc_curve(false_positive_rate, true_positive_rate)

pred = model.predict_proba(X_test)[:,1]
roc_auc_score(y_test, pred, average='macro', sample_weight=None)

In [ ]:
model = XGBClassifier()

params = {
          'max_depth': [3,5,7],
          'eta': [0.01, 0.05, 0.1],
          'n_estimators': [100, 500, 1000],
          'colsample_bytree': [0.3, 0.7]}

clf = GridSearchCV(estimator=model,
                   param_grid=params,
                   scoring='accuracy',
                   verbose=1)

clf.fit(X,y)
print("Best parameters:", clf.best_params_)
print("Best F1: ", (-clf.best_score_))

# Performance metrics
grid_best = clf.best_estimator_.predict(X_train)
errors = abs(grid_best - y_train)

# Calculate mean absolute percentage error (MAPE)
mape = np.mean(100 * (errors/y_train))

# Calculate and display accuracy
accuracy = 100 - mape

print('The best model from grid-search has an accuracy of', round(accuracy, 2),'%')

In [ ]:
params = {'max_depth':[3,5,6,8],
          'learning_rate':[0.01,0.05,0.1,0.2],
          'subsample':[0.4,1.0,0.1],
          'colsample_bytree':[0.4,1.0,0.1],
          'colsample_bylevel':[0.4,1.0,0.1],
          'n_estimators':[100,500,1000]}
  
model = XGBClassifier()

clf = RandomizedSearchCV(estimator=model,
                         param_distributions = params,
                         scoring='neg_mean_squared_error',
                         n_iter=25,
                         verbose=1)

clf.fit(X,y)

print("Best parameters:", clf.best_params_)
print("Lowest RMSE: ", (-clf.best_score_)**(1/2.0))

In [ ]:
model = XGBClassifier(objective="binary:logistic", eta=0.01, max_depth=6, min_child_weight=1, 
                      scale_pos_weight=1, gamma=0, colsample_bytree=1, nthread=4,
                      early_stopping_round=5)
model,Summary_XGB = PrintResults(model, X_train, y_train, 'XGB')
y_train_XGB = pd.Series(model.predict(X_train), name = "XGB")
y_test_XGB = pd.Series(model.predict(X_test), name = "XGB")

In [ ]:
y_scores = model.predict_proba(X_test)
y_scores = y_scores[:,1]

false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_scores)

def plot_roc_curve(false_positive_rate, true_positive_rate, label=None):
    plt.plot(false_positive_rate, true_positive_rate, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'r', linewidth=4)
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate (FPR)', fontsize=16)
    plt.ylabel('True Positive Rate (TPR)', fontsize=16)

plt.figure(figsize=(14, 7))
plot_roc_curve(false_positive_rate, true_positive_rate)

pred = model.predict_proba(X_test)[:,1]
roc_auc_score(y_test, pred, average='macro', sample_weight=None)

In [ ]:
model = SVC()

params = {'C':[0.1,1,10,100,1000],
          'gamma':[1,0.1,0.01,0.001,0.0001],
          'kernel':['rbf']}

clf = GridSearchCV(SVC(),
                   params,
                   refit = True,
                   verbose = 3)

clf.fit(X,y)
print("Best parameters:", clf.best_params_)
print("Best F1: ", (-clf.best_score_))
print(clf.best_estimator_)

In [ ]:
model = SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf', max_iter=-1,
    probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)
model, Summary_SVM = PrintResults(model, X_train_scaled, y_train_scaled, 'SVM')
y_train_SVM = pd.Series(model.predict(X_train_scaled), name = "SVM")
y_test_SVM = pd.Series(model.predict(X_test_scaled), name = 'SVM')

In [ ]:
y_scores = model.predict_proba(X_test_scaled)
y_scores = y_scores[:,1]

false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_scores)

def plot_roc_curve(false_positive_rate, true_positive_rate, label=None):
    plt.plot(false_positive_rate, true_positive_rate, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'r', linewidth=4)
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate (FPR)', fontsize=16)
    plt.ylabel('True Positive Rate (TPR)', fontsize=16)

plt.figure(figsize=(14, 7))
plot_roc_curve(false_positive_rate, true_positive_rate)

pred = model.predict_proba(X_test_scaled)[:,1]
roc_auc_score(y_test, pred, average='macro', sample_weight=None)

In [ ]:
model = KNeighborsClassifier()

params = {'n_neighbors':[3,5,11,19],
          'weights':['uniform','distance'],
          'metric':['euclidean','manhattan']}

clf = GridSearchCV(model,
                   params,
                   scoring='accuracy',
                   verbose=1)

clf.fit(X,y)
clf.best_score_
clf.best_estimator_
clf.best_params_
print("Best parameters:", clf.best_params_)
print("Best F1: ", (-clf.best_score_))

In [ ]:
model = KNeighborsClassifier(n_neighbors = 11, metric = 'manhattan', weights = 'uniform')
model, Summary_KNN = PrintResults(model, X_train_scaled, y_train, 'KNN')
y_train_KNN = pd.Series(model.predict(X_train_scaled), name = "KNN")
y_test_KNN = pd.Series(model.predict(X_test_scaled), name="KNN")

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

y_scores = model.predict_proba(X_test_scaled)
y_scores = y_scores[:,1]

false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_scores)

def plot_roc_curve(false_positive_rate, true_positive_rate, label=None):
    plt.plot(false_positive_rate, true_positive_rate, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'r', linewidth=4)
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate (FPR)', fontsize=16)
    plt.ylabel('True Positive Rate (TPR)', fontsize=16)

plt.figure(figsize=(14, 7))
plot_roc_curve(false_positive_rate, true_positive_rate)

pred = model.predict_proba(X_test_scaled)[:,1]
roc_auc_score(y_test, pred, average='macro', sample_weight=None)